In [0]:
# ops/metrics_notebook.py
from pyspark.sql.functions import current_timestamp, lit
import argparse, os

parser = argparse.ArgumentParser()
parser.add_argument("--notify_only", action="store_true")
parser.add_argument("--catalog", default=os.getenv("CATALOG", "main"))
parser.add_argument("--schema",  default=os.getenv("SCHEMA",  "metrics"))
parser.add_argument("--domain",  default="unknown")
args = parser.parse_args()

spark.sql(f"CREATE CATALOG IF NOT EXISTS {args.catalog}")
spark.sql(f"CREATE SCHEMA  IF NOT EXISTS {args.catalog}.{args.schema}")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {args.catalog}.{args.schema}.job_run_facts
(run_ts TIMESTAMP, domain STRING, run_id STRING, status STRING, rows BIGINT)
USING delta
""")

# In a real run you'd compute rows from gold_serving, here we mock it.
df = spark.createDataFrame(
    [(None, args.domain, os.getenv("RUN_ID","${job.id}-${task.run_id}"), "SUCCESS", 0)],
    "run_ts TIMESTAMP, domain STRING, run_id STRING, status STRING, rows BIGINT"
).withColumn("run_ts", current_timestamp())

df.write.mode("append").saveAsTable(f"{args.catalog}.{args.schema}.job_run_facts")
print("Metrics appended.")
